# BQML - Linear Regression

In [20]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
PROJECT_NUM = !gcloud projects list --filter="$PROJECT_ID" --format="value(PROJECT_NUMBER)"
PROJECT_NUM = PROJECT_NUM[0]
LOCATION = 'us-central1'
REGION = "us-central1"

# VERTEX_SA = '934903580331-compute@developer.gserviceaccount.com'
VERTEX_SA = 'jt-vertex-sa@hybrid-vertex.iam.gserviceaccount.com'

print(f"PROJECT_ID: {PROJECT_ID}")
print(f"PROJECT_NUM: {PROJECT_NUM}")
print(f"LOCATION: {LOCATION}")
print(f"REGION: {REGION}")
print(f"VERTEX_SA: {VERTEX_SA}")

PROJECT_ID: hybrid-vertex
PROJECT_NUM: 934903580331
LOCATION: us-central1
REGION: us-central1
VERTEX_SA: jt-vertex-sa@hybrid-vertex.iam.gserviceaccount.com


In [21]:
VERSION='jtv4'

In [22]:
REGION = 'us-central1'
EXPERIMENT = 'forecasting-2'
SERIES = f'{VERSION}-forecasting'

BQ_PROJECT = PROJECT_ID
BQ_DATASET = SERIES.replace('-','_')
BQ_TABLE = 'forecasting-1'

viz_limit = 12

VERTEX_AI_MODEL_ID='v2_mlr'
MODEL_VERSION='v1'
XAI_FLAG="TRUE"

EXPERIMENT_NAME = f"nyc_{BQ_DATASET}"
print(f'EXPERIMENT_NAME: {EXPERIMENT_NAME}')

EXPERIMENT_NAME: nyc_jtv2_forecasting


In [23]:
from google.cloud import bigquery

import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta

from google.cloud import aiplatform as vertex_ai

In [24]:
bq = bigquery.Client(project=PROJECT_ID)

vertex_ai.init(
    project=PROJECT_ID, 
    location=REGION,
    # credentials=credentials
)

## review time series data

In [25]:
# CUSTOMIZE
TARGET_COLUMN = 'num_trips'
TIME_COLUMN = 'starttime'
SERIES_COLUMN = 'start_station_name'
COVARIATE_COLUMNS = ['avg_tripduration', 'pct_subscriber', 'ratio_gender', 'capacity'] # could be empty

In [26]:
query = f"""
    WITH
        SPLIT AS (
            SELECT splits, min({TIME_COLUMN}) as mindate, max({TIME_COLUMN}) as maxdate
            FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
            GROUP BY splits
        ),
        TRAIN AS (
            SELECT mindate as start_date
            FROM SPLIT
            WHERE splits ='TRAIN'
        ),
        VAL AS (
            SELECT mindate as val_start
            FROM SPLIT
            WHERE splits = 'VALIDATE'
        ),
        TEST AS (
            SELECT mindate as test_start, maxdate as end_date
            FROM SPLIT
            WHERE splits = 'TEST'
        )
    SELECT * EXCEPT(pos) FROM
    (SELECT *, ROW_NUMBER() OVER() pos FROM TRAIN)
    JOIN (SELECT *, ROW_NUMBER() OVER() pos FROM VAL)
    USING (pos)
    JOIN (SELECT *, ROW_NUMBER() OVER() pos FROM TEST)
    USING (pos)
"""
keyDates = bq.query(query).to_dataframe()
keyDates

,start_date,val_start,test_start,end_date
0,2013-07-01,2016-09-03,2016-09-17,2016-09-30


In [27]:
query = f"""
    SELECT {SERIES_COLUMN}, {TIME_COLUMN}, {TARGET_COLUMN}, splits,
        {', '.join(COVARIATE_COLUMNS)}
    FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
    ORDER by {SERIES_COLUMN}, {TIME_COLUMN}
"""
rawSeries = bq.query(query).to_dataframe()

# Train regression model

In [28]:
# CUSTOMIZE
forecast_granularity = 'DAY'
forecast_horizon = 14
forecast_test_length = 14
#forecast_val_length = 14

In [29]:
query = f"""
    CREATE OR REPLACE MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_mlr`
    OPTIONS
      (model_type = 'linear_reg',
       input_label_cols = ['{TARGET_COLUMN}'],
       -- Vertex AI fields
       model_registry="vertex_ai", 
       vertex_ai_model_id='{VERTEX_AI_MODEL_ID}',
       vertex_ai_model_version_aliases=['{MODEL_VERSION}', 'experimental']
       -- enable_global_explain={XAI_FLAG}
      ) AS
    SELECT {TIME_COLUMN}, {TARGET_COLUMN},
        {', '.join(COVARIATE_COLUMNS)}
    FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
    WHERE splits in ('TRAIN','VALIDATE')
        AND {SERIES_COLUMN} = 'Central Park S & 6 Ave'
"""
print(query)


    CREATE OR REPLACE MODEL `hybrid-vertex.jtv2_forecasting.forecasting-1_mlr`
    OPTIONS
      (model_type = 'linear_reg',
       input_label_cols = ['num_trips']
      ) AS
    SELECT starttime, num_trips,
        avg_tripduration, pct_subscriber, ratio_gender, capacity
    FROM `hybrid-vertex.jtv2_forecasting.forecasting-1_prepped`
    WHERE splits in ('TRAIN','VALIDATE')
        AND start_station_name = 'Central Park S & 6 Ave'



In [30]:
job = bq.query(query)
job.result()
print(job.state, (job.ended-job.started).total_seconds())

DONE 21.146


## Review the input features
* Reference for `ML.FEATURE_INFO`

In [31]:
query = f"""
    SELECT *
    FROM ML.FEATURE_INFO(MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_mlr`)
"""
featureInfo = bq.query(query).to_dataframe()
featureInfo.head()

,input,min,max,mean,median,stddev,category_count,null_count,dimension
0,starttime,NaN,NaN,NaN,NaN,NaN,942,0,<NA>
1,avg_tripduration,336.625,34597.615385,1930.967077,1827.845815,1457.060583,<NA>,0,<NA>
2,pct_subscriber,0.000,1.000000,0.502280,0.470588,0.216679,<NA>,0,<NA>
3,ratio_gender,0.000,14.000000,0.941306,0.573864,1.416410,<NA>,22,<NA>
4,capacity,73.000,73.000000,73.000000,73.000000,0.000000,<NA>,0,<NA>


In [32]:
query = f"""
    SELECT *
    FROM ML.TRAINING_INFO(MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_mlr`)
"""
trainingInfo = bq.query(query).to_dataframe()
trainingInfo.head()

,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,2,200.898354,11245.422644,0.8,2507
1,0,1,2222.959311,12849.695984,0.4,2987
2,0,0,14040.935031,20287.378013,0.2,2154


# Forecast Evaluation

## Forecast Metrics

In [33]:
query = f"""
    SELECT *
    FROM ML.EVALUATE(
        MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_mlr`,
        (
            SELECT {TIME_COLUMN}, {TARGET_COLUMN},
                {', '.join(COVARIATE_COLUMNS)}
            FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
            WHERE splits = 'TEST'
                AND {SERIES_COLUMN} = 'Central Park S & 6 Ave'
        )
    )
"""
metrics = bq.query(query).to_dataframe()
metrics

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,244.036399,73735.74786,1.763317,276.33459,-3.002883,0.230104


In [34]:
metrics['mean_absolute_error'][0]#.values

244.03639872292794

In [35]:
metrics['explained_variance'][0]

0.23010438426930913

In [36]:
from datetime import datetime

# create run name
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
EXPERIMENT_RUN_NAME = f"run-{TIMESTAMP}"

# log params and metrics to dicts
params = {}
# params["budget_hrs"] = MILLI_NODE_HRS
params["horizon"] = forecast_horizon
# params["context_window"] = CONTEXT_WINDOW
params["model_type"] = "mlr"

metrics_dict = {}
metrics_dict["MAE"] = metrics['mean_absolute_error'][0]
metrics_dict["RMSE"] = metrics['mean_squared_error'][0]
metrics_dict["RMSLE"] = metrics['mean_squared_log_error'][0]
metrics_dict["MedianSE"] = metrics['median_absolute_error'][0]
metrics_dict["r2_score"] = metrics['r2_score'][0]
metrics_dict["explained_variance"] = metrics['explained_variance'][0]

# # Create and log experiment
vertex_ai.init(experiment=EXPERIMENT_NAME.replace("_","-"))

with vertex_ai.start_run(EXPERIMENT_RUN_NAME) as my_run:
    my_run.log_metrics(metrics_dict)
    my_run.log_params(params)

    vertex_ai.end_run()

Associating projects/934903580331/locations/us-central1/metadataStores/default/contexts/nyc-jtv2-forecasting-run-20230515110722 to Experiment: nyc-jtv2-forecasting


INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/934903580331/locations/us-central1/metadataStores/default/contexts/nyc-jtv2-forecasting-run-20230515110722 to Experiment: nyc-jtv2-forecasting


### Forecast Time series 

In [37]:
query = f"""
    SELECT *
    FROM ML.PREDICT(
        MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_mlr`,
        (
            SELECT {TIME_COLUMN}, {TARGET_COLUMN},
                {', '.join(COVARIATE_COLUMNS)}
            FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
            WHERE splits = 'TEST'
                AND {SERIES_COLUMN} = 'Central Park S & 6 Ave'
        )
        )
"""
forecast = bq.query(query).to_dataframe()
forecast
# print(query)

,predicted_num_trips,starttime,num_trips,avg_tripduration,pct_subscriber,ratio_gender,capacity
0,76.710803,2016-09-27,264,1879.386364,0.526515,0.660377,73
1,79.583619,2016-09-26,312,1325.112179,0.496795,0.752809,73
2,40.622148,2016-09-30,60,1316.350000,0.733333,1.400000,73
3,63.047728,2016-09-20,327,1170.850153,0.599388,0.958084,73
4,70.192270,2016-09-22,351,1320.150997,0.561254,0.763819,73
5,113.717057,2016-09-17,607,1688.902801,0.286656,0.305376,73
6,77.665410,2016-09-23,354,1527.000000,0.514124,0.710145,73
7,68.134705,2016-09-19,135,1098.281481,0.570370,0.824324,73
8,106.188210,2016-09-24,399,1676.949875,0.338346,0.312500,73
9,113.486474,2016-09-25,402,1721.203980,0.288557,0.305195,73
